In [ ]:
%load_ext autoreload
%autoreload 2

# general
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from src.economics import run_economic_analysis

data = run_economic_analysis.step_load_data(verbose=False)
aligned = run_economic_analysis.step_align_data(data, verbose=True)

# # total_tourism_value_gdf.groupby("country")["approx_price_corrected"].sum().sort_values(ascending=False).head(10)
# total_tourism_value_gdf["approx_price_corrected"].sum() / 1e9
# # TODO: this (load_tourism_data) isn't matching with my implementation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

────────────────────────────────────────────────────────────
💰 TOURISM VALUE DATA LOADED
────────────────────────────────────────────────────────────
  Polygons: 63,180
  Countries: 113
  Value column: 'approx_price_corrected'

  📊 Value Statistics:
     Total: $28.37 billion
     Mean per polygon: $449,043
     Median per polygon: $0
     Non-zero polygons: 31,483 (49.8%)

  🏆 Top 10 Countries by Value:
      1. Indonesia: $3,097M (10.9%)
      2. Mexico: $3,000M (10.6%)
      3. Thailand: $2,410M (8.5%)
      4. United States of America: $2,388M (8.4%)
      5. Australia: $2,176M (7.7%)
      6. China: $1,872M (6.6%)
      7. Philippines: $1,385M (4.9%)
      8. Japan: $1,178M (4.2%)
      9. Malaysia: $1,149M (4.0%)
     10. Maldives: $1,085M (3.8%)
────────────────────────────────────────────────────────────

✓ Loaded GDP data: 257 countries for year 2022

──────────────────────────────────────

In [ ]:
exp_col = "Sum of reef-associated visitor expenditure (1000)"
total = (
    pd.read_csv(
        "/Users/rt582/Library/CloudStorage/OneDrive-UniversityofCambridge/cambridge/phd/Paper_Conferences/reef_cover_economics/data/economics/tourism/A1_summary_data.csv"
    )[exp_col]
    .replace({",": ""}, regex=True)
    .pipe(pd.to_numeric, errors="coerce")
    * 1000
)
total.sum()

np.int64(35851038000)

In [27]:
data["tourism"].gdf.groupby("country")["approx_price_corrected"].sum().sort_values(
    ascending=False
)


country
Indonesia                   3.097453e+09
Mexico                      2.999883e+09
Thailand                    2.410154e+09
United States of America    2.387729e+09
Australia                   2.176084e+09
                                ...     
New Zealand                 0.000000e+00
Pitcairn Is.                0.000000e+00
Scarborough Reef            0.000000e+00
Spratly Is.                 0.000000e+00
Guatemala                   0.000000e+00
Name: approx_price_corrected, Length: 113, dtype: float64